# Model Training

In [12]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch import optim
from tqdm import tqdm
import os

from code.train_model import ModelTrainer
from code.function import load_train_data, load_test_data

from model.gru_base import GRUModel
from model.gru_patch import GRUPatchModel

## load data

load the processed data and split the train set, test set and valid set

In [13]:
X = np.load('data/processed_data/X.npy')
ret5 = np.load('data/processed_data/ret5.npy')
ret10 = np.load('data/processed_data/ret10.npy')

sample_datetime = np.load('data/processed_data/sample_datetime.npy', allow_pickle=True)
sample_stock = np.load('data/processed_data/sample_stock.npy', allow_pickle=True)
print(X.shape, ret5.shape, ret10.shape, sample_datetime.shape, sample_stock.shape)

(5272, 2498, 6) (5272, 2498) (5272, 2498) (2498,) (5272,)


In [14]:
train_end = int(len(sample_datetime) * 0.9)
valid_end = int(len(sample_datetime))

seq_len = 40

X_train, X_valid = X[:, :train_end], X[:, train_end-seq_len:valid_end]
y_train, y_valid = ret10[:, :train_end], ret10[:, train_end-seq_len:valid_end]
train_date, valid_date = sample_datetime[:train_end], sample_datetime[train_end:valid_end]

print(X_train.shape, y_train.shape, train_date.shape)
print(X_valid.shape, y_valid.shape, valid_date.shape)

(5272, 2248, 6) (5272, 2248) (2248,)
(5272, 290, 6) (5272, 290) (250,)


In [15]:
x1_train, x1_test, y_train, y_test = load_train_data(X_train, y_train)
BATCH_SIZE = 5000

class Newdataset(Dataset):
    def __init__(self, data1, label) -> None:
        super().__init__()
        self.data1 = data1.astype(np.float32)
        self.label = label.astype(np.float32)

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        return self.data1[index], self.label[index]

train_ds = Newdataset(x1_train, y_train)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

test_ds = Newdataset(x1_test, y_test)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

## set model

set up the model trainer

In [16]:
model1 = GRUModel()
model2 = GRUPatchModel()

optimizer1 = optim.Adam(model1.parameters(), lr=0.005)
optimizer2 = optim.Adam(model2.parameters(), lr=0.005)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

modeltrainer1 = ModelTrainer(model1, optimizer1, device, 'gru')
modeltrainer2 = ModelTrainer(model2, optimizer2, device, 'gru_patch')

if not os.path.exists(f'data/saved_model'):
    os.makedirs(f'data/saved_model')

train model

In [17]:
# modeltrainer1.fit(train_dl, test_dl, 'data')
# modeltrainer2.fit(train_dl, test_dl, 'data')

## predict factor

use the model in valid set and get the factor output

In [18]:
# load model
model1 = torch.load('data/saved_model/gru.pt')
model2 = torch.load('data/saved_model/gru_patch.pt')

optimizer1 = optim.Adam(model1.parameters(), lr=0.005)
optimizer2 = optim.Adam(model2.parameters(), lr=0.005)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

modeltrainer1 = ModelTrainer(model1, optimizer1, device, 'gru')
modeltrainer2 = ModelTrainer(model2, optimizer2, device, 'gru_patch')

In [19]:
def predict_valid_set(X_valid, y_valid, modeltrainer, valid_date, sample_stock):
    fac_1 = pd.DataFrame(np.nan * np.zeros((X_valid.shape[0], len(valid_date)-10)))
    i_panel = 0
    for i in tqdm(range(len(valid_date)-10)):
        x1_test, y_test, nonan_index = load_test_data(X_valid[:, i:i+seq_len, :], y_valid[:, i:i+seq_len])
        test_ds = Newdataset(x1_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=len(x1_test))

        y_pred = modeltrainer.predict(test_dl)
        fac_1.iloc[nonan_index, i_panel] = y_pred[:, -1]
        i_panel += 1
    fac_1.columns = valid_date[:i_panel]
    fac_1.index = sample_stock
    return fac_1

In [20]:
fac1 = predict_valid_set(X_valid, y_valid, modeltrainer1, valid_date, sample_stock)
fac2 = predict_valid_set(X_valid, y_valid, modeltrainer2, valid_date, sample_stock)

100%|██████████| 240/240 [01:48<00:00,  2.21it/s]


In [21]:
if not os.path.exists(f'data/saved_factor/'):
    os.makedirs(f'data/saved_factor/')

fac1.to_csv('data/saved_factor/fac1.csv')
fac2.to_csv('data/saved_factor/fac2.csv')